In [62]:
import pandas as pd
from pathlib import Path

In [63]:
# Load this paquet file data/movies.parquet
df = pd.read_parquet("../data/movies.parquet")

In [64]:
# Filter out rows with missing values
df.shape

(608940, 9)

In [65]:
# Check for missing values
df.isnull().sum()

title_id                   0
title                      2
original_title             2
release_year               0
runtime_minutes       166443
genre                  70424
imdb_rating           279360
vote_count            279360
production_country      7928
dtype: int64

In [66]:
# Remove missing values
df = df.dropna()

In [67]:
# Check for missing values
df.isnull().sum()

title_id              0
title                 0
original_title        0
release_year          0
runtime_minutes       0
genre                 0
imdb_rating           0
vote_count            0
production_country    0
dtype: int64

In [54]:
df.rename(columns={
    'tconst': 'title_id',
    'primaryTitle': 'title',
    'originalTitle': 'original_title',
    'year': 'release_year',
    'runtimeMinutes': 'runtime_minutes',
    'genres': 'genre',
    'rating': 'imdb_rating',
    'votes': 'vote_count',
    'country': 'production_country'
}, inplace=True)


In [34]:
# Change all year to integer and remove NA and missing rows from year
df['release_year'] = df['release_year'].astype('Int64')
df = df.dropna(subset=['release_year'])

In [ ]:
# Convert runtime minutes to integer 
df['runtime_minutes'] = df['runtime_minutes'].astype('Int64')

# Convert vote count to integer 
df['vote_count'] = df['vote_count'].astype('Int64')

In [69]:
# Save as parquet in data directory
output_path = data_dir / "movies.parquet"
df.to_parquet(output_path, index=False)
print(f"Movies data saved to {output_path}")

Movies data saved to ../data/movies.parquet


In [68]:
missing_values = df.isnull().sum()
print("Missing values in each column:")
print(missing_values[missing_values > 0])


Missing values in each column:
Series([], dtype: int64)
